# SunPy and HelioPy Tutorial

Viewing solar data with SunPy
---
In the first half of this tutorial, we'll look at how to search for, download, and plot remote sensing data using SunPy.

Searching and Downloading Data with SunPy
---
In order to search for some data, we have to select a time range and an instrument to search for. In addition, here we also specify the wavelength of interest.

Searching for data
---
Using the above defined search parameters, Fido can be used to search for data. For more information on searching for and downloading data see https://docs.sunpy.org/en/stable/guide/acquiring_data/index.html

Downloading data
---
The results from a search can also be downloaded.

``Fido.fetch`` returns a list of the local location of the downloaded files.

SunPy `Map` and `MapSequence`
---
Now we have downloaded some data, we can load it and plot it. ``sunpy.map.Map`` can be used to load any ``.fits`` file, creating a ``Map`` object. We can then take a look at the image stored by calling ``map.peek()``.

## `MapSequence`

## `Map`

## Coordinates

![SunPy Graph](http://docs.sunpy.org/en/latest/_images/graphviz-2fafba2dd42c9a3e49915c57f04a039dda5cc9a4.svg)

## Plotting

Identifying a coronal hole
---
In the above AIA 193 image, there is a dark patch in the middle of the disc. This is a coronal hole, which is the source of the fastest solar wind.

This observation can be connected to in-situ measurements of the solar wind at 1 AU by looking for the fast solar wind stream that emmenates from this coronal hole. Because the solar wind takes a finite amount of time to propagate from the Sun to Earth, we first do an order of magnitude estimate of this delay.

Using astropy units
---
To calculate the propagation delay we can use the ``astropy.units`` module. This provides an extension of normal numbers and arrays, and allows units to be attached. All the unit mathematics is calculated automatically, avoiding the need to keep track of specific units.

Assume the solar wind is relesed from the surface of the Sun, so the propagation distance is $D_{sun} - R_{sun}$

Take a typical fast solar wind speed of 500 km/s

Calculate tne propagation time, and convert it to units of days

Downloading and importing in-situ data
---
The ``heliopy.data`` module can be used to download and import a wide range of in situ datasets from various heliospheric missions. In this example we use data from OMNI, which provides measurements of the solar wind at the orbit of the Earth.

The data is stored in a [SunPy TimeSeries](https://docs.sunpy.org/en/latest/code_ref/timeseries.html) object:

We can print the available columns in this object:

Plotting in-situ data
---
Matplotlib can be used to plot the downloaded data. In this example we plot the solar wind speed and the magnetic field clock angle, to see different polarity solar wind streams.

Improving figure formatting
---

We can improve this plot a little with some better formatting.

We also add a vertical line where the stream is fast stream is predicted to have arrived using the above back-of-the-envelope calculation. We can see that it lines up nicely with a fast solar wind stream that has speeds of 500 - 600 km/s.

Save a copy of the figure
---